In [1]:
import os, openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["LANGCHAIN_PROJECT"] = "financial-agent"

## Setup tools

In [3]:
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools import PolygonLastQuote, PolygonTickerNews, PolygonFinancials, PolygonAggregates

polygon = PolygonAPIWrapper()
tools = [
    PolygonLastQuote(api_wrapper=polygon),
    PolygonTickerNews(api_wrapper=polygon),
    PolygonFinancials(api_wrapper=polygon),
    PolygonAggregates(api_wrapper=polygon)
]

In [4]:
for tool in tools:
    print(tool.name)

polygon_last_quote
polygon_ticker_news
polygon_financials
polygon_aggregates



## Agent

In [5]:
from langchain_openai.chat_models import AzureChatOpenAI
llm  = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4-32k",
    model="gpt-4-32k",
    streaming=True,
    temperature=0
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        As a Financial Research Assistant, your role is to provide latest financial insights about companies and help individuals make informed investment decisions. 
        
        You have access to 4 tools from Polygon.io API that lets you query the latest financial data from all US stock exchanges. The tools are:
        1. polygon_last_quote: Get the latest quote for ticker
        2. polygon_ticker_news: Get the latest news for ticker
        3. polygon_financials: Get the financials for ticker
        4. polygon_aggregates: Get the aggregates for ticker   

        You may need one or more tools to answer a user query. You can use the tools in any order depending on the user query. 
        
        Conversaion History:
        {chat_history}
        """                 
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "User Input: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

prompt.pretty_print()

================================ System Message ================================


        As a Financial Research Assistant, your role is to provide latest financial insights about companies and help individuals make informed investment decisions. 
        
        You have access to 4 tools from Polygon.io API that lets you query the latest financial data from all US stock exchanges. The tools are:
        1. polygon_last_quote: Get the latest quote for ticker
        2. polygon_ticker_news: Get the latest news for ticker
        3. polygon_financials: Get the financials for ticker
        4. polygon_aggregates: Get the aggregates for ticker   

        You may need one or more tools to answer a user query. You can use the tools in any order depending on the user query. 
        
        Conversaion History:
        {chat_history}
        

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message 

LangChain has several abstractions to make working with agents easy.

### AgentAction
This is a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)

### AgentFinish
This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an output key containing a string that is the agent's response.

### Intermediate Steps
These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a List[Tuple[AgentAction, Any]]. Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string.

### `AgentExecutor` is a built-in class that provides the `run_agent` functionality

```console
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "retrieve": retrieve, 
            "calculate_total": calculate_total
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

```

Also adds additional functionalities such as logging, error handling for tools and also the entire agent.

In [14]:
import random
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="shiva-test" + str(random.randint(1, 10000)),
    user_id="shivac"
    )
cosmos.prepare_cosmos()

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(k=10, return_messages=True,memory_key="chat_history",chat_memory=cosmos)

In [21]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent_runnable, tools=tools, verbose=False, memory=memory) 
# verbose=True to see the agent's internal state i.e. which tools are being used by the agent to answer the user query

In [16]:
result = agent_executor.invoke({"input": "How can you help me?"}) 



> Entering new AgentExecutor chain...
As a Financial Research Assistant, I can provide you with the latest financial insights about companies to help you make informed investment decisions. I can fetch the latest price of a stock, get the latest news for a stock, fetch fundamental financials from balance sheets, income statements, and cash flow statements for a stock ticker, and fetch aggregate bars (stock prices) for a ticker. How can I assist you today?

> Finished chain.


{'input': 'How can you help me?',
 'chat_history': [HumanMessage(content='How can you help me?'),
  AIMessage(content='As a Financial Research Assistant, I can provide you with the latest financial insights about companies to help you make informed investment decisions. I can fetch the latest price of a stock, get the latest news for a stock, fetch fundamental financials from balance sheets, income statements, and cash flow statements for a stock ticker, and fetch aggregate bars (stock prices) for a ticker. How can I assist you today?')],
 'output': 'As a Financial Research Assistant, I can provide you with the latest financial insights about companies to help you make informed investment decisions. I can fetch the latest price of a stock, get the latest news for a stock, fetch fundamental financials from balance sheets, income statements, and cash flow statements for a stock ticker, and fetch aggregate bars (stock prices) for a ticker. How can I assist you today?'}

In [17]:
result = agent_executor.invoke({"input": "Why is CRM stock down so much? Was something wrong with its earnings? Could you be specific what went wrong."})



> Entering new AgentExecutor chain...

Invoking: `polygon_ticker_news` with `{'query': 'CRM'}`


[{"id": "uEi6tw4UW3Pg2wkgc1_HkGq8V_hUtizhgYCa4bF9V5A", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "My Top 24 Stocks to Buy Now in June (2024)", "author": "newsfeedback@fool.com (Parkev Tatevosian, CFA)", "published_utc": "2024-06-03T16:06:03Z", "article_url": "https://www.fool.com/investing/2024/06/03/my-top-24-stocks-to-buy-now-in-june-2024/", "tickers": ["NVDA", "HD", "GOOGL", "DIS", "KO", "CRM", "NFLX", "AMZN", "TGT", "META", "EBAY", "V", "AMD", "NKE", "SIX", "GOOG", "BABA", "UBER", "PYPL", "SNAP", "ZM", "CHWY", "FVRR"], "image_url": "https://g.foolcdn.com/editorial/images/779392/new-york-stock-exchange-building.jpg", "description": "Whether you are looking for artificial in

In [20]:
print(result["output"])

Based on the latest news and financial data for Salesforce.com (CRM), here are some key points:

1. The company's stock has been under pressure due to a weak outlook. According to an article by The Motley Fool, investors are worried about the future prospects of the customer relationship management giant. The article titled "Salesforce Stock Sinks on Weak Outlook. Time to Buy the Dip?" discusses this in detail. You can read it [here](https://www.fool.com/investing/2024/06/02/salesforce-stock-sinks-on-weak-outlook-time-to-buy/).

2. Another article by The Motley Fool titled "Is Salesforce Stock a Buy Now?" mentions that the cloud software giant has disappointed a lot of investors. You can read it [here](https://www.fool.com/investing/2024/06/03/is-salesforce-stock-a-buy-now/).

3. The financial data shows that the company's net income for the fiscal year 2024 was $4.136 billion. However, the net income for the last quarter (Q4) of the fiscal year was $1.446 billion, which indicates a de

In [24]:
import time

print("I am a Financial Research Assistant! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = agent_executor.invoke({"input": query})
    end_time = time.time()
    execution_time = end_time - start_time
    print("\nUser: {0}".format(query))
    print("\nAssistant: {0}".format(result['output']))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")

I am a Financial Research Assistant! How can I assist you?


User: How can you help me?

Assistant: As a Financial Research Assistant, I can provide you with the latest financial insights about companies to help you make informed investment decisions. I can fetch the latest price of a stock, get the latest news for a stock, fetch fundamental financials from balance sheets, income statements, and cash flow statements for a stock ticker, and fetch aggregate bars (stock prices) for a ticker. How can I assist you today?
	Time taken to respond: 9 seconds

User: Microsoft stock has been down the last week despite good releases at the BUILD conference. What happened? Is it a good buy?

Assistant: Based on the latest news and financial data for Microsoft (MSFT), here are some key points:

1. The Motley Fool article titled "The Best Stocks to Invest $1,000 in Right Now" mentions Microsoft as one of the top stocks to consider. You can read it [here](https://www.fool.com/investing/2024/06/03/the-

## Gradio Interface

In [25]:
import gradio as gr

def run_agent(query):
    result = agent_executor.invoke({"input": query})
    return result['output']

ui = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Financial Agent",
    description="Financial Data Explorer: Leveraging Advanced API Tools for Market Insights"
)

ui.launch()

c:\Users\shchitt\AppData\Local\anaconda3\envs\langgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
